In [1]:
# Version required for this code: opencv-python==3.4.2.16
from __future__ import print_function
import cv2
import numpy as np
import imutils

#Important Threshold Parameter
BEST_MATCH = 0.75

In [2]:
#Function that aligns the two images using RANSAC
def image_alignment_func(image_1, image_2):

    # Conversion of images to grayscale
    gray_image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    gray_image_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
  
    # Detect SIFT features and compute descriptors for the images
    desc = cv2.xfeatures2d.SIFT_create()
    keypoints_image_1, descriptors_image_1 = desc.detectAndCompute(gray_image_1, None)
    keypoints_image_2, descriptors_image_2 = desc.detectAndCompute(gray_image_2, None)
    
    # Convert keypoints to float values for further processing
    keypoints_image_1 = np.float32([i.pt for i in keypoints_image_1])
    keypoints_image_2 = np.float32([i.pt for i in keypoints_image_2])
    
    # Match features for the images
    matcher_for_image = cv2.DescriptorMatcher_create("BruteForce")
    matches_found = matcher_for_image.knnMatch(descriptors_image_1, descriptors_image_2, 2)

    best_matches = []
    for val in matches_found:
        if len(val) == 2 and val[0].distance < val[1].distance * BEST_MATCH:
            best_matches.append((val[0].trainIdx, val[0].queryIdx))

    # Get the location of the top matches
    points_image_1 = np.zeros((len(best_matches), 2), dtype=np.float32)
    points_image_2 = np.zeros((len(best_matches), 2), dtype=np.float32)

    for i, match in enumerate(best_matches):
        points_image_1 = np.float32([keypoints_image_1[i] for (_,i) in best_matches])
        points_image_2 = np.float32([keypoints_image_2[i] for (i,_) in best_matches])
    
    # Find homography
    homo_index, mask = cv2.findHomography(points_image_1, points_image_2, cv2.RANSAC, 4.0)
    
    # Warping the images using homography factor
    width = image_1.shape[1] + image_2.shape[1];
    image_1_registered = cv2.warpPerspective(image_1, homo_index, (width, image_1.shape[0]))
    image_1_registered[0:image_2.shape[0], 0:image_2.shape[1]] = image_2
    
    # Draw the matches for the 2 images
    height_1 = image_1.shape[0]
    height_2 = image_2.shape[0]
    width_1 = image_1.shape[1]
    width_2 = image_2.shape[1]
    
    matches = np.zeros((max(height_1, height_2), width_1 + width_2, 3), dtype="uint8")
    matches[0:height_1, 0:width_1] = image_1
    matches[0:height_2, width_1:] = image_2
    
    for ((trainIdx, queryIdx), s) in zip(best_matches, mask):
        if s == 1:
            point_1 = (int(keypoints_image_1[queryIdx][0]), int(keypoints_image_1[queryIdx][1]))
            point_2 = (int(keypoints_image_2[trainIdx][0]) + width_1, int(keypoints_image_2[trainIdx][1]))
            cv2.line(matches, point_1, point_2, (255, 0, 255), 1)

    cv2.imwrite("Match.jpg",matches)
    
    return image_1_registered


In [3]:
# Reference image (Image 1)
image_1_fname = "input1.png"
print("Reference image (Image 1) : ", image_1_fname)
image_reference = cv2.imread(image_1_fname, cv2.IMREAD_COLOR)

# Image to be aligned (Image 2)
image_2_fname = "input2.png"
print("Image to be aligned (Image 2) : ", image_2_fname);  
image_align = cv2.imread(image_2_fname, cv2.IMREAD_COLOR)
  
print("RANSAC for image alignment:")

# Registered image will be stored in image_registered 
# Estimated homography will be stored in est_homography
image_registered = image_alignment_func(image_align, image_reference)
  
# Save the aligned image to the disk
result_filename = "aligned.jpg"
cv2.imwrite(result_filename, image_registered)
print("Aligned Image saved to the disk: ", result_filename);   

Reference image (Image 1) :  input1.png
Image to be aligned (Image 2) :  input2.png
RANSAC for image alignment:
Aligned Image saved to the disk:  aligned.jpg
